# Import

In [ ]:
import os
import numpy as np
import random
import time
from PIL import Image
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.python.slim.nets.inception_v3 import inception_v3
from tensorflow.contrib.slim.python.slim.nets.inception_v3 import inception_v3_arg_scope

# Build graph

In [ ]:
g = tf.get_default_graph()

## Define preprocessing

In [ ]:
with tf.name_scope('Preprocessing'):
    input_im = tf.placeholder(tf.uint8, shape=[None, None, None, 3], name='input_im')
    resized_im = tf.image.resize_bilinear(tf.image.convert_image_dtype(
        tf.convert_to_tensor(input_im) ,dtype=tf.float32), [299,299], name='resized_im')
    normalized_im = tf.mul(tf.sub(resized_im, 0.5), 2.0, name='normalized_im')

## Load Inception-v3 graph from slim

In [ ]:
with slim.arg_scope(inception_v3_arg_scope()):
    logits, end_points = inception_v3(normalized_im, num_classes=1001, is_training=False)
slim_variables = [v for v in tf.global_variables() if v.name.startswith('InceptionV3/')]

## Create own branch

In [ ]:
with tf.name_scope('Own'):
    y = tf.placeholder("float")
    x = tf.reshape(g.get_tensor_by_name('InceptionV3/Logits/SpatialSqueeze:0'),(1,1001))
    w = tf.Variable(tf.random_normal([int(x.get_shape()[-1]),300], stddev=float('1e-5')), name='weights')
    b = tf.Variable(tf.random_normal([1,300]), name='bias')
    y_pred = tf.add(tf.matmul(x, w), b, name='y_pred')
    cost = tf.reduce_sum(tf.square(y-y_pred),name='cost')
    cost_summary = tf.summary.scalar('cost', cost)

## Define training

In [ ]:
with tf.name_scope('Optimization'):
    global_step = tf.Variable(0, trainable=False, name='global_step')
    init_learning_rate = tf.constant(float('1e-5'), name='init_learning_rate')
    learning_rate = tf.train.exponential_decay(
        init_learning_rate, global_step, 100, 0.95, staircase=True, name='learning_rate')

    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate, name='Optimizer')
    
    variables_to_train = [w,b]
    variables_to_train.extend([v for v in tf.trainable_variables() if
                               v.name.startswith('InceptionV3/Logits') and
                               (v.name.endswith('weights:0') or v.name.endswith('biases:0'))])
    variables_to_train.extend([v for v in tf.trainable_variables() if
                               v.name.startswith('InceptionV3/Mixed_7c') and
                               (v.name.endswith('weights:0') or v.name.endswith('beta:0'))])
    
    train_step = optimizer.minimize(cost, var_list=variables_to_train, global_step=global_step, name='train_step')

## Save graph

In [ ]:
timestamp = time.strftime('%Y%m%d-%H%M', time.localtime())
tf.summary.FileWriter('./runs/'+timestamp+'/graph/', graph=g)

# Read data

In [ ]:
# Images
train_image_path = '../../../data/train2014/'
train_image_list = os.listdir(train_image_path)
val_image_path = '../../../data/val2014/'
val_image_list = os.listdir(val_image_path)

# Caption dictionaries
train_dict = np.load('../../../data/word2vec_train.npy').item()
val_dict = np.load('../../../data/word2vec_val.npy').item()

# Labels
labels = open('../../../data/ilsvrc2012_labels.txt', 'r').readlines()

# Train model

In [ ]:
# Tried to follow
# https://github.com/tensorflow/models/tree/master/slim
# http://stackoverflow.com/questions/35274457/inceptionv3-and-transfer-learning-with-tensorflow/40709836#40709836
with tf.Session() as sess:
    
    # Initialize variables
    saver = tf.train.Saver(var_list=slim_variables)
    saver.restore(sess, "../../../data/inception-2016-08-28/inception_v3.ckpt")
    sess.run([w.initializer, b.initializer])
    sess.run([global_step.initializer])
#    uninitialized = list(tf.get_variable(name) for name in
#                         sess.run(tf.report_uninitialized_variables(tf.global_variables())))
#    input(uninitialized)
#    sess.run(tf.initialize_variables(uninitialized_variables))
    
    # Initialize writers
    train_writer = tf.summary.FileWriter('./runs/'+timestamp+'/sums/train/', flush_secs=20)
    val_writer = tf.summary.FileWriter('./runs/'+timestamp+'/sums/val/', flush_secs=20)
    
    # Train
    print('TRAINING')
    for i, im in enumerate(train_image_list[:1000]):
        try:
            # Choose one of the five captions randomly
            r = random.randrange(len(train_dict[im]))
            y_temp = train_dict[im][r].reshape((1,300))

            # Form input and feed through
            image = [np.array(Image.open(train_image_path + im))]
            train_writer.add_summary(sess.run(cost_summary,{input_im:image, y:y_temp}), i)
            sess.run(train_step,feed_dict={input_im:image, y:y_temp})
            print(i, end=' ')

    #        # Get prediction
    #        class_pred = np.argmax(sess.run(
    #            g.get_tensor_by_name('InceptionV3/Predictions/Softmax:0'),{input_im:image}))
    #        print(labels[class_pred])

            # Evaluate every 25th step
            if i % 25 == 0:
                val_cost = 0
                val_set_size = 10
                for j, val_im in enumerate(val_image_list[:val_set_size]):
                    val_image = [np.array(Image.open(val_image_path + val_im))]
                    y_val = val_dict[val_im][0].reshape((1,300))
                    val_cost += sess.run(cost,{input_im:val_image, y:y_val})
                val_writer.add_summary(sess.run(cost_summary,feed_dict={cost:val_cost/val_set_size}), i)
        except (IndexError, ValueError):
            print('-/{}'.format(i), end=' ')
            
    # Save
    os.mkdir('./runs/'+timestamp+'/checkpoint/')
    new_saver = tf.train.Saver()
    save_path = new_saver.save(sess, './runs/'+timestamp+'/checkpoint/model.ckpt')
    print('\n\nTrained model saved to {}'.format(save_path))